# Search Repeat Cycle of Satellite from its TLE
## Satellite must be polar orbit

In [1]:
import ephem

In [2]:
# Get Ephemeris Data
def getEphem(satellite, date):
  satellite.compute(date)
  latitude  = satellite.sublat / ephem.degree
  longitude = satellite.sublong / ephem.degree
  hight     = satellite.elevation
  mmotion   = satellite._n
  obperiod  = 1 / mmotion
  return (latitude, longitude, hight, mmotion, obperiod)

In [3]:
# Search date with Latitude = lat
# satellite: ephem object
# date: latitude must be near lat at specified date
# lat: target latitude
# eps : small enough value
def lat0date(satellite, date, lat, eps):
  d1 = date
  satellite.compute(d1)
  l1  = satellite.sublat / ephem.degree
 
  d2 = d1 + eps
  satellite.compute(d2)
  l2  =satellite.sublat / ephem.degree

  a = (l2 - l1) / eps
  b = l1 - a * d1

  dt   = (lat - b) / a
  
  return dt

In [4]:
import datetime
def jday2str(jday):
    (year, month, day, hour, minute, second) = ephem.Date(jday).tuple()
    second = int(second)
    dt = datetime.datetime(year, month, day, hour, minute, second)
    return dt.isoformat().replace('T', ' ')

In [5]:
# TLE: TLE of satellite
# datestr: Initial date (string)
# maxlat: max error of latitude[deg]
# maxlong: max error of longitude[deg]
# maxdays: max days of search
def searchRepeatCycle(TLE, datestr, maxlat, maxlong, maxdays):
    # Initial Date
    dt0 = dt = ephem.Date(datestr)

    # Calculate Initial Ephemeris
    (line1, line2, line3) = TLE.split("\n")
    satellite = ephem.readtle(line1, line2, line3)
    (latitude0, longitude0, hight0, mmotion0, obperiod0) = getEphem(satellite, dt)

    # Search Repeat Cycle
    eps      = obperiod0 / 360 / 10
    latlen   = 40009 # Circumference - meridional [Km]
    longlen  = 40075 # Circumference - quatorial  [Km]

    dt = dt0
    print("                                         Lat(+N) diff[deg]   diff[Km] |   Long(+E)  diff[deg]   diff[Km]")
    for d in range(int(maxdays * mmotion0)):
      (latitude, longitude, _, _, _) = getEphem(satellite, dt)
      difflat   = latitude  - latitude0
      difflong  = longitude - longitude0

      if abs(difflat) < maxlat and abs(difflong) <maxlong:
    #    dtstr= str(ephem.localtime(ephem.Date(dt)))[0:19]
        dtstr =jday2str(dt)
        days  = dt - dt0
        difflatlen  = difflat  / 360 * latlen
        difflonglen = difflong / 360 * longlen
        print("[%s = %6.2f(days)] %10.4f %+10.4f %+10.4f | %10.4f %+10.4f %+10.4f" % (dtstr, days, latitude, difflat, difflatlen, longitude, difflong, difflonglen))

      # update dt
      dt = lat0date(satellite, dt + obperiod0, latitude0, eps)

In [9]:
# Search Repeat Cycle for each TLE and datestr
def searchRepeatCycles(TLEs, datestrs, maxlat, maxlong, maxdays):
    for TLE in TLEs:
        print("\n" + TLE + "\n")
        for datestr in datestrs:
            searchRepeatCycle(TLE, datestr, maxlat, maxlong, maxdays)
            print("")    

In [10]:
# INPUT DATA
# Two Line Element
TLE1 = '''WORLDVIEW-1 (WV-1)
1 32060U 07041A   18258.02295190  .00000790  00000-0  35109-4 0  9993
2 32060  97.3879  16.4069 0002228  57.6683  54.8987 15.24397549611548'''

TLE2 = '''WORLDVIEW-1 (WV-1)      
1 32060U 07041A   18258.81766689  .00000958  00000-0  41975-4 0  9990
2 32060  97.3884  17.1911 0002170  64.8045  86.2416 15.24400435611664'''

TLEs = (TLE1, TLE2)

# dates to search
datestrs = ("2018-09-18 10:46:01", "2018-09-18 10:41:01", "2018-09-18 10:36:01")

# Search Condition
maxlat   = 1.0 # max error of latitude[deg]
maxlong  = 1.0 # max error of longitude[deg]
maxdays  = 180 # max days of search

searchRepeatCycles(TLEs, datestrs, maxlat, maxlong, maxdays)


WORLDVIEW-1 (WV-1)
1 32060U 07041A   18258.02295190  .00000790  00000-0  35109-4 0  9993
2 32060  97.3879  16.4069 0002228  57.6683  54.8987 15.24397549611548

                                         Lat(+N) diff[deg]   diff[Km] |   Long(+E)  diff[deg]   diff[Km]
[2018-09-18 10:46:01 =   0.00(days)]    -0.1840    +0.0000    +0.0000 |    40.9467    +0.0000    +0.0000
[2018-10-05 10:47:27 =  17.00(days)]    -0.1840    +0.0000    +0.0001 |    40.5894    -0.3573   -39.7726
[2018-10-18 10:42:41 =  30.00(days)]    -0.1840    +0.0000    +0.0000 |    41.7862    +0.8395   +93.4514
[2018-10-22 10:48:26 =  34.00(days)]    -0.1840    +0.0000    +0.0000 |    40.3508    -0.5959   -66.3349
[2018-11-04 10:43:17 =  47.00(days)]    -0.1840    +0.0000    +0.0000 |    41.6392    +0.6925   +77.0940
[2018-11-08 10:48:55 =  51.00(days)]    -0.1840    +0.0000    +0.0000 |    40.2321    -0.7145   -79.5430
[2018-11-21 10:43:25 =  64.00(days)]    -0.1840    +0.0000    +0.0000 |    41.6122    +0.6655   +74.0823